In [53]:
import pandas as pd
import re

In [54]:
year = "1961"

In [55]:
df_stops = pd.read_csv(f"{year}-stops-localised-final-west.csv")  

In [56]:
df_stops['stop_name'] = df_stops['stop_name'].astype(str).str.strip()
df_stops['location'] = df_stops['location'].astype(str).str.strip()

In [57]:
df_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   stop_name   930 non-null    object
 1   type        930 non-null    object
 2   location    930 non-null    object
 3   in_lines    930 non-null    object
 4   identifier  156 non-null    object
 5   east-west   930 non-null    object
dtypes: object(6)
memory usage: 43.7+ KB


In [58]:
df_stops = df_stops.fillna('').groupby(['stop_name', 'type', 'location']).agg({
    'in_lines': ', '.join,
    'identifier': ', '.join
}).reset_index()

# Reset the index
df_stops.reset_index(inplace=True)
df_stops.rename(columns={'index': 'stop_id'}, inplace=True)

df_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   stop_id     930 non-null    int64 
 1   stop_name   930 non-null    object
 2   type        930 non-null    object
 3   location    930 non-null    object
 4   in_lines    930 non-null    object
 5   identifier  930 non-null    object
dtypes: int64(1), object(5)
memory usage: 43.7+ KB


In [59]:
df_line = pd.read_csv(f"../initial/line_data_{year}_west.csv")  

In [60]:
# Replace U+00a0 with regular whitespace in the entire DataFrame
df_line.replace('\u00a0', ' ', regex=True, inplace=True)

In [61]:
df_line.head()

,Unnamed: 0,line_name,Stops,Frequency_7-30,Length (time),type,east-west,year
0,1,2,Bernauer Strasse Ecke Wolliner Strasse - Volta...,15.0,60.0,strassenbahn,West,1961
1,2,3,Grüntaler Straße Ecke Osloer Strasse - U-Bhf. ...,10.0,87.0,strassenbahn,West,1961
2,7,15,Daimlerstrasse - Bhf. Marienfelde - Körtingstr...,15.0,36.0,strassenbahn,West,1961
3,9,26,Spreewaldplatz - Reichenberger Strasse - Gloga...,15.0,30.0,strassenbahn,West,1961
4,10,27,Spreewaldplatz - Hermannplatz - U-Bhf. Boddins...,15.0,30.0,strassenbahn,West,1961


In [62]:
# Assuming df is your DataFrame and 'stop_id' is the column you want to modify
df_stops['stop_id'] = year + df_stops['stop_id'].astype(str)
df_stops.head(2)

,stop_id,stop_name,type,location,in_lines,identifier
0,19610,4. Ring,bus,"52.42250630689789, 13.2869490566549",A53,
1,19611,Adamstrasse,bus,"52.521029117218866, 13.19780699419913","A34,A35",


In [63]:
df_line['line_name'] = df_line['line_name'].astype(str).str.strip()
df_line['Stops'] = df_line['Stops'].astype(str).str.strip()
df_line['year'] = df_line['year'].astype(int)
df_line['Frequency_7-30	'] = pd.to_numeric(df_line['Frequency_7-30'], errors='coerce').fillna(0).astype(int)
df_line['Length (time)'] = pd.to_numeric(df_line['Length (time)'], errors='coerce').fillna(0).astype(int)

In [64]:
# Define a function to remove leftover parentheses from a column
def remove_double_whitespace(text):
    text = text.replace("   ", " ")
    text = text.replace("  ", " ")
    text = text.replace("–", "-")
    text = text.replace(" - ", " - ")
    return text

# Apply the function to the 'line_stops' column
df_line['Stops'] = df_line['Stops'].apply(remove_double_whitespace)
df_line['Stops'] = df_line['Stops'].str.replace(r'\s*–\s*', ' - ', regex=True)

In [65]:
mask = df_line['Stops'].str.contains(" - ").fillna(True)
none_df = df_line[~mask]
none_df

,Unnamed: 0,line_name,Stops,Frequency_7-30,Length (time),type,east-west,year,Frequency_7-30\t


In [66]:
# Filter rows where 'line_stops' is not a string
non_string_rows = df_line[~df_line['Stops'].apply(lambda x: isinstance(x, str))]
non_string_rows

,Unnamed: 0,line_name,Stops,Frequency_7-30,Length (time),type,east-west,year,Frequency_7-30\t


In [67]:
### Creating related tables

In [68]:
def extract_first_start_stop(string):
    stations = string.split(" - ")
    first_station = stations[0]
    last_station = stations[-1]
    return f"{first_station}<> {last_station}"

In [69]:
def create_line_df(df):
    line_df = pd.DataFrame({
        'line_id': range(1, 1+len(df)),
        'year': df['year'],
        'line_name': df['line_name'],
        'type': df["type"],
        "start_stop": df['Stops'].apply(extract_first_start_stop),
        "Length (time)": df['Length (time)'],
        "east_west": df['east-west'],
        "Frequency_7-30": df['Frequency_7-30']
    })
    return line_df

In [70]:
line_df = create_line_df(df_line)

In [71]:
line_df.head(2)

,line_id,year,line_name,type,start_stop,Length (time),east_west,Frequency_7-30
0,1,1961,2,strassenbahn,Bernauer Strasse Ecke Wolliner Strasse<> Goten...,60,West,15.0
1,2,1961,3,strassenbahn,Grüntaler Straße Ecke Osloer Strasse<> Elsenst...,87,West,10.0


In [72]:
# Assuming df is your DataFrame and 'stop_id' is the column you want to modify
line_df['line_id'] = year + line_df['line_id'].astype(str)
line_df.head(2)

,line_id,year,line_name,type,start_stop,Length (time),east_west,Frequency_7-30
0,19611,1961,2,strassenbahn,Bernauer Strasse Ecke Wolliner Strasse<> Goten...,60,West,15.0
1,19612,1961,3,strassenbahn,Grüntaler Straße Ecke Osloer Strasse<> Elsenst...,87,West,10.0


In [73]:
def create_line_stops_df(df):
    line_stops = df['Stops'].str.split(' - ', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='stop_name')

    line_stops['stop_order'] = line_stops.groupby('index').cumcount()
    #index starts from 0 so it looks like 1 row is missing but this is not true

    # Clean the 'Stop Name' column by removing whitespace and non-breaking spaces
    line_stops['stop_name'] = line_stops['stop_name'].str.replace(u'\xa0', ' ').str.strip()

    # reset index so that it can be used for foreign key

    return line_stops

In [74]:
line_stops_df = create_line_stops_df(df_line)

In [75]:
for index, row in line_stops_df.iterrows():
    line_stops_df.at[index, "index"] = row["index"] + 1
line_stops_df.rename(columns={'index': 'line_id'}, inplace=True)

line_stops_df

,line_id,stop_name,stop_order
0,1,Bernauer Strasse Ecke Wolliner Strasse,0
1,1,Voltastrasse Ecke Watt Strasse,1
2,1,Weddingplatz,2
3,1,Rathenower Strasse Ecke Perleberger Strasse,3
4,1,Alt-Moabit Ecke Rathenower Strasse,4
...,...,...,...
1305,91,Lankwitz,15
1306,91,Lichterfelde Ost,16
1307,91,Lichterfelde Süd,17
1308,92,Kladow,0


In [76]:
# Assuming df is your DataFrame and 'stop_id' is the column you want to modify
line_stops_df['line_id'] = year + line_stops_df['line_id'].astype(str)
line_stops_df.head(2)

,line_id,stop_name,stop_order
0,19611,Bernauer Strasse Ecke Wolliner Strasse,0
1,19611,Voltastrasse Ecke Watt Strasse,1


In [77]:
# get a list of unique Line IDs in the line_stops DataFrame
line_ids = line_stops_df['line_id'].unique()

# loop through each unique Line ID
for line_id in line_ids:
    # get the first and last Stop Name for this Line ID
    first_stop_name = line_stops_df.loc[line_stops_df['line_id'] == line_id, 'stop_name'].iloc[0]
    last_stop_name = line_stops_df.loc[line_stops_df['line_id'] == line_id, 'stop_name'].iloc[-1]
    
    # get the corresponding Start-Stop value from line_df for this Line ID
    start_stop = line_df.loc[line_df['line_id'] == line_id, 'start_stop'].iloc[0]
    
    # check if the first and last Stop Names are contained in the Start-Stop value
    if first_stop_name not in start_stop or last_stop_name not in start_stop:
        print(f"Quality control check failed for line_id {line_id}.")

# I checked that this works using the following code:
# line_stops_df.loc[0, "stop_name"] = "test"
# and got the right result: Quality control check failed for line_id 1001.


In [78]:
def add_type(line_stops, line_df):
    # Assuming line_id is the common column between line_stops and line_df
    merged_df = pd.merge(line_stops, line_df[['line_id', 'type', "line_name"]], on='line_id', how='left')
    
    # Rename the 'type' column from line_df to 'type_from_line_df' to avoid conflicts
    merged_df.rename(columns={'type': 'type'}, inplace=True)
    
    # Drop the 'type_from_line_df' column if it's not needed in the final result
    # merged_df.drop(columns=['type_from_line_df'], inplace=True)
    
    return merged_df

In [79]:
line_stops_df = add_type(line_stops_df, line_df)

In [80]:
line_stops_df.loc[120:130]

,line_id,stop_name,stop_order,type,line_name
120,19618,Lynastrasse Ecke Neuendorfer Strasse,11,strassenbahn,54
121,19618,Hohenzollernring,12,strassenbahn,54
122,19618,Johannesstift,13,strassenbahn,54
123,19619,Nollendorfplatz,0,strassenbahn,55
124,19619,Wittenbergplatz,1,strassenbahn,55
125,19619,Bhf. Zoo,2,strassenbahn,55
126,19619,U-Bhf. Ernst-Reuter-Platz,3,strassenbahn,55
127,19619,Cauerstrasse,4,strassenbahn,55
128,19619,Richard-Wagner-Platz,5,strassenbahn,55
129,19619,Luisenplatz,6,strassenbahn,55


In [81]:
df_stops

,stop_id,stop_name,type,location,in_lines,identifier
0,19610,4. Ring,bus,"52.42250630689789, 13.2869490566549",A53,
1,19611,Adamstrasse,bus,"52.521029117218866, 13.19780699419913","A34,A35",
2,19612,Adolf-Scheidt-Platz,strassenbahn,"52.478194856980814, 13.37836902238447",96,
3,19613,Afrikanische Str.,u-bahn,"52.560027777778,13.334633333333",C I,Q559239
4,19614,Afrikanische Strasse,bus,"52.54630154020319, 13.343855247543928",A16,
...,...,...,...,...,...,...
925,1961925,Zoologischer Garten,u-bahn,"52.5058669,13.3330236","A I,B I",Q4024764
926,1961926,Zossener Strasse,bus,"52.49160676567746, 13.394443897793668",A28,
927,1961927,Zossener Strasse Ecke Blücherstrasse,strassenbahn,"52.49622054069677, 13.395106811633234",95,
928,1961928,Zum Heckeshorn,bus,"52.43307505069834, 13.164008736441371",A51,


In [82]:
def add_fk(line_stops_df, df_stops):
    # Create a new dataframe with the Stop Name and Stop ID columns
    stop_id_df = df_stops[['stop_name', 'stop_id', 'type', 'in_lines']]

    # Merge the line_stops_df and stop_id_df dataframes based on matching stop names and line_name condition
    line_stops_df = line_stops_df.merge(stop_id_df,
                                        left_on=['stop_name', 'type'],
                                        right_on=['stop_name', 'type'],
                                        how='left')

    # Filter the rows based on the condition that line_name is contained in in_lines
    line_stops_df = line_stops_df[line_stops_df.apply(lambda row: str(row['line_name']) in str(row['in_lines']), axis=1)]

    return line_stops_df

# Assuming line_stops_df and df_stops are your dataframes
# Replace 'stop_name', 'stop_id', 'type', and 'in_lines' with the actual column names you have

line_stops_df = add_fk(line_stops_df, df_stops)


In [83]:
# Calculate the difference between consecutive 'stop_order' values
line_stops_df['diff'] = line_stops_df['stop_order'].diff()

# Use the following line to drop rows where the 'diff' column is 0.0
line_stops_df = line_stops_df[line_stops_df['diff'] != 0.0]

# Identify faulty rows where the difference is not 1 digit behind
faulty_rows = line_stops_df[(line_stops_df['diff'] != 1) & (line_stops_df['stop_order'] != 0)]
faulty_rows

,line_id,stop_name,stop_order,type,line_name,stop_id,in_lines,diff
91,19616,Wildmeisterdamm,12,strassenbahn,47,1961890,47,2.0


In [138]:
faulty_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   line_id     0 non-null      object 
 1   stop_name   0 non-null      object 
 2   stop_order  0 non-null      int64  
 3   type        0 non-null      object 
 4   line_name   0 non-null      object 
 5   stop_id     0 non-null      object 
 6   in_lines    0 non-null      object 
 7   diff        0 non-null      float64
dtypes: float64(1), int64(1), object(6)
memory usage: 0.0+ bytes


In [139]:
# convert 'Stop ID' column to numeric values, coercing errors to NaN
line_stops_df['stop_id'] = pd.to_numeric(line_stops_df['stop_id'], errors='coerce')

# check if all values in 'Stop ID' column are numeric
if line_stops_df['stop_id'].notnull().all():
    print("All values in 'stop_id' column are numeric")
else:
    print("There are non-numeric values in 'stop_id' column")
    print(line_stops_df[line_stops_df['stop_id'].isnull()])


All values in 'stop_id' column are numeric


In [140]:
line_stops_df.drop(['line_name', "in_lines", "type", "diff"], axis=1, inplace=True)

In [142]:
line_stops_df.to_csv(f"line_stops_{year}-final.csv")
line_df.to_csv(f"line_df_{year}-final.csv")
df_stops.to_csv(f"stops_df_{year}-final.csv")